In [1]:
import requests
import json
import time
import pymongo

In [2]:
db = pymongo.MongoClient().local.ratemyprofs

In [ ]:
counter = 0

for i in range(1,210):
    query = "http://www.ratemyprofessors.com/filter/professor/?department=&institution=California+State+University%2C+Northridge&page="+str(i)+"&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid=163"

    try:
        page = requests.get(query)
        jsonpage = json.loads(page.content)
        professorlist = jsonpage['professors']

    if len(professorlist)>0:
        for prof in professorlist:
            if db.find_one({"tid":int(prof['tid'])}) is None:
                db.insert(prof)
    print("page "+str(i)+" "+'\xF0\x9F\x98\x8F')
    
    except:
        pass

time.sleep(1)

http://www.ratemyprofessors.com/search.jsp?queryBy=schoolId&schoolName=University+of+Texas+at+Austin&schoolID=1255&queryoption=TEACHER

In [10]:
# correcting university names
import pandas as pd
university_rankings = pd.read_json('university_rankings.json')

In [13]:
# correcting professor names
university_rankings.index

Index(['Boston University', 'California Institute of Technology',
       'Carnegie Mellon University', 'Colorado School of Mines',
       'Columbia University (Fu Foundation)', 'Cornell University',
       'Duke University (Pratt)', 'Emory University-Georgia Tech University',
       'Georgia Institute of Technology', 'Harvard University',
       'Iowa State University', 'Johns Hopkins University (Whiting)',
       'Louisiana State University--Baton Rouge',
       'Massachusetts Institute of Technology', 'Michigan State University',
       'North Carolina State University',
       'Northwestern University (McCormick)', 'Ohio State University',
       'Oregon State University',
       'Pennsylvania State University--University Park',
       'Princeton University', 'Purdue University--West Lafayette',
       'Rice University (Brown)', 'Stanford University',
       'Texas A&M University--College Station',
       'Texas Tech University (Whitacre)',
       'University of California--Berkeley

In [14]:
# Dictionary of school ids (keys) that map to corrected school name
sids = {124 : 'Boston University',
        181: 'Carnegie Mellon University', 
        'NA': 'Colorado School of Mines',
        278: 'Columbia University', 
        298: 'Cornell University',
        1350:'Duke University', 
        340: 'Emory University',
        361: 'Georgia Institute of Technology', 
        'NA': 'Harvard University',
        452: 'Iowa State University', 
        464: 'Johns Hopkins University',
        3071: 'Louisiana State University',
        580: 'Massachusetts Institute of Technology', 
        601: 'Michigan State University',
        685: 'North Carolina State University',
        709:'Northwestern University', 
        724: 'The Ohio State University',
        742:'Oregon State University',
        758:'Pennsylvania State University',
        780:'Princeton University', 
        783: 'Purdue University - West Lafayette',
        799: 'Rice University', 
        953: 'Stanford University',
       1003: 'Texas A&M University at College Station',
       1011: 'Texas Tech University',
       1072: 'University of California Berkeley', 
       1073:'University of California Davis',
       1079: 'University of California San Diego',
       1077: 'University of California Santa Barbara',
       1087: 'University of Colorado - Boulder', 
       1094: 'University of Delaware',
       1100: 'University of Florida', 
       1112:'University of Illinois at Urbana-Champaign',
       1117: 'University of Kansas', 
       1270: 'University of Maryland',
       1258: 'University of Michigan',
       1257: 'University of Minnesota - Twin Cities',
       1249: 'University of Nebraska - Lincoln', 
       1596: 'University of Oklahoma',
       1275:'University of Pennsylvania',
       1381: 'University of Southern California',
       1385: 'University of Tennessee--Knoxville (Tickle)',
       1255: 'University of Texas at Austin', 
       3963:'University of Tulsa',
       1530: 'University of Washington', 
       1256: 'University of Wisconsin - Madison',
       4282: 'University of Wyoming', 
       1349: 'Virginia Tech', 
       1222: 'Yale University'}

In [16]:
with open('sids.json', 'w') as f:
    json.dump(sids, f)